# 🎬 Streaming Insights | User Behavior Analysis on Streaming Platforms

**Business context:** Streaming platforms see high consumption but non-trivial content abandonment.
This notebook analyzes user behavior (device, time-of-day, content type, plan) to surface patterns
that drive session completion and informs tactics to reduce churn and increase engagement.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

sns.set(style='whitegrid')
np.random.seed(42)
print('✅ Environment successfully configured!')


In [ ]:
for d in ['data', 'images', 'reports']:
    os.makedirs(d, exist_ok=True)
print('📁 Ensured folders: data/, images/, reports/')


## Data Loading / Generation
If `data/streaming_data.csv` does not exist, a synthetic dataset (~10k rows) will be generated.
Embedded patterns to reflect realistic behavior:
- **Mobile penalty** for long content (>45min)
- **Short documentaries (<30min)** have the highest completion rate
- **Lower engagement around 10–11 PM**
- **Free plan** has the lowest average completion rate


In [ ]:
csv_path = os.path.join('data', 'streaming_data.csv')
n = 10_000

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f'📥 Loaded existing dataset: {csv_path} | shape={df.shape}')
else:
    # --- Synthetic data generation with business patterns ---
    content_types = np.random.choice(['series','movie','documentary'], size=n, p=[0.5,0.3,0.2])
    device_types  = np.random.choice(['mobile','tv','desktop','tablet'], size=n, p=[0.45,0.35,0.1,0.1])
    user_plans    = np.random.choice(['free','basic','premium'], size=n, p=[0.35,0.4,0.25])
    age_groups    = np.random.choice(['18-24','25-34','35-44','45-54','55+'], size=n, p=[0.18,0.32,0.22,0.18,0.10])
    regions       = np.random.choice(['NA','LATAM','EU','APAC'], size=n, p=[0.5,0.15,0.2,0.15])

    # Duration by content type (minutes)
    base_duration = {
        'series':      (35, 12),   # mean, std
        'movie':       (105, 25),
        'documentary': (50, 20)
    }
    durations = np.clip(np.array([
        np.random.normal(base_duration[ct][0], base_duration[ct][1]) for ct in content_types
    ]), 5, 240)

    # Start times across a day
    start_hours = np.random.choice(np.arange(0,24), size=n, p=np.array([
        # hour distribution favoring evenings
        0.02,0.015,0.01,0.008,0.008,0.01,0.015,0.02,
        0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,
        0.10,0.09,0.08,0.07,0.05,0.035,0.02,0.015
    ]) / np.sum([
        0.02,0.015,0.01,0.008,0.008,0.01,0.015,0.02,
        0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,
        0.10,0.09,0.08,0.07,0.05,0.035,0.02,0.015
    ]))
    start_time_str = [f"{h:02d}:{np.random.randint(0,60):02d}:{np.random.randint(0,60):02d}" for h in start_hours]

    # Watch dates last 90 days
    today = datetime.now().date()
    days_back = np.random.randint(0, 90, size=n)
    watch_dates = [str(today - timedelta(days=int(d))) for d in days_back]

    # Baseline completion via Beta, then apply business effects
    # Start with slightly different baselines by content type
    baseline_map = {'series': (2.5, 2.0), 'movie': (2.2, 2.3), 'documentary': (2.8, 1.9)}
    baseline = np.array([np.random.beta(*baseline_map[ct]) for ct in content_types])

    # Apply penalties/boosts
    long_mask = durations > 45
    mobile_mask = (device_types == 'mobile')
    doc_mask = (content_types == 'documentary')
    short_doc_mask = doc_mask & (durations < 30)
    hour_10_pm_mask = (start_hours == 22)  # 10 PM
    free_mask = (user_plans == 'free')

    completion = baseline.copy()
    # Mobile penalty on long content
    completion[long_mask & mobile_mask] -= 0.15
    # Short documentaries boost
    completion[short_doc_mask] += 0.15
    # 10 PM dip
    completion[hour_10_pm_mask] -= 0.10
    # Additional dip for free plan
    completion[free_mask] -= 0.07
    # Slight TV boost
    completion[(device_types=='tv')] += 0.03

    # Add a little noise
    completion += np.random.normal(0, 0.03, size=n)
    completion = np.clip(completion, 0, 1)

    watched = np.clip(completion * durations + np.random.normal(0, 3, size=n), 0, None)

    df = pd.DataFrame({
        'user_id': np.random.randint(100000, 999999, size=n),
        'content_type': content_types,
        'duration_minutes': durations.round(0).astype(int),
        'watched_minutes': watched.round(0).astype(int),
        'device_type': device_types,
        'watch_date': watch_dates,
        'start_time': start_time_str,
        'age_group': age_groups,
        'region': regions,
        'user_plan': user_plans
    })

    df.to_csv(csv_path, index=False)
    print(f'🧪 Generated synthetic dataset and saved to {csv_path} | shape={df.shape}')


In [ ]:
display(df.head())
display(df.describe(include='all').T)


## Feature Engineering
We will create: `completion_rate`, `start_hour`, and `long_session`.

In [ ]:
df['watch_date'] = pd.to_datetime(df['watch_date'])
df['completion_rate'] = (df['watched_minutes'] / df['duration_minutes']).clip(0,1)
df['start_hour'] = pd.to_datetime(df['start_time'], format='%H:%M:%S').dt.hour
df['long_session'] = df['watched_minutes'] > 90
print('🧩 Features created: completion_rate, start_hour, long_session')
display(df.head())


## Exploratory Data Analysis (EDA)
Charts generated below will also be saved under `images/`.

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
order = ['series','movie','documentary']
sns.boxplot(data=df, x='content_type', y='completion_rate', order=order, ax=ax)
ax.set_title('Completion Rate by Content Type')
ax.set_xlabel('Content Type')
ax.set_ylabel('Completion Rate (0-1)')
plt.ylim(0,1)
plt.tight_layout()
img1 = os.path.join('images','content_type_boxplot.png')
plt.savefig(img1, dpi=120)
plt.show()
print(f'🖼️ Saved: {os.path.abspath(img1)} | exists={os.path.exists(img1)}')


In [ ]:
required_cols = {'start_hour','content_type','completion_rate'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f'Missing columns: {missing}. Run feature engineering first.')

pivot = df.pivot_table(
    index='start_hour',
    columns='content_type',
    values='completion_rate',
    aggfunc='mean'
)
fig, ax = plt.subplots(figsize=(9,6))
sns.heatmap(pivot.sort_index(), annot=False, fmt='.2f', cmap='YlGnBu', ax=ax, vmin=0, vmax=1)
ax.set_title('Average Completion Rate by Hour × Content Type')
ax.set_xlabel('Content Type')
ax.set_ylabel('Start Hour')
plt.tight_layout()
img2 = os.path.join('images','hourly_engagement_heatmap.png')
plt.savefig(img2, dpi=120)
plt.show()
print(f'🖼️ Saved: {os.path.abspath(img2)} | exists={os.path.exists(img2)}')


In [ ]:
avg_by_device = df.groupby('device_type', as_index=False)['completion_rate'].mean().sort_values('completion_rate')
fig, ax = plt.subplots(figsize=(7,5))
sns.barplot(data=avg_by_device, x='device_type', y='completion_rate', ax=ax)
ax.set_title('Average Completion Rate by Device Type')
ax.set_xlabel('Device Type')
ax.set_ylabel('Completion Rate (0-1)')
plt.ylim(0,1)
plt.tight_layout()
plt.show()
avg_by_device


## Strategic Insights
The following metrics quantify key patterns observed in the analysis.

In [ ]:
lines = []

# 1) Mobile vs TV penalty on long content (>45 min)
long_mask = df['duration_minutes'] > 45
mobile_long = df[long_mask & (df['device_type']=='mobile')]['completion_rate'].mean()
tv_long = df[long_mask & (df['device_type']=='tv')]['completion_rate'].mean()
penalty_pct = (tv_long - mobile_long) * 100
lines.append(f"- Mobile vs TV (content >45 min): mobile={mobile_long:.2%}, tv={tv_long:.2%}, penalty={penalty_pct:.1f} pp")

# 2) Short documentaries (<30 min)
short_doc = df[(df['content_type']=='documentary') & (df['duration_minutes']<30)]['completion_rate'].mean()
overall = df['completion_rate'].mean()
lift_pct = (short_doc - overall) * 100
lines.append(f"- Short documentaries (<30 min) avg completion={short_doc:.2%} (lift vs overall: {lift_pct:.1f} pp)")

# 3) Worst hour overall and for free plan
hourly_overall = df.groupby('start_hour')['completion_rate'].mean().sort_values()
worst_hour = int(hourly_overall.index[0])
worst_hour_val = hourly_overall.iloc[0]

hourly_free = df[df['user_plan']=='free'].groupby('start_hour')['completion_rate'].mean().sort_values()
worst_hour_free = int(hourly_free.index[0])
worst_hour_free_val = hourly_free.iloc[0]
lines.append(f"- Lowest engagement window overall: {worst_hour}:00 with {worst_hour_val:.2%}")
lines.append(f"- Lowest engagement (free plan): {worst_hour_free}:00 with {worst_hour_free_val:.2%}")

summary_path = os.path.join('reports','summary.md')
with open(summary_path, 'w', encoding='utf-8') as f:
    f.write('# Key Insights\n\n')
    for ln in lines:
        f.write(ln + '\n')

print('\n'.join(lines))
print(f'📝 Insights saved to {summary_path}')


## Business Recommendations
Retention and personalization levers derived from insights above.

In [ ]:
recs = [
    'Prioritize shorter content in low-engagement windows; adjust recommendation ranking after 22:00.',
    'Improve mobile UX for long sessions (resume points, offline downloads, network-adaptive bitrate).',
    'Target free-plan cohorts with lightweight content and upgrade prompts for heavy watchers.',
    'Run A/B tests on recommendation algorithms and watch-next carousels for low-completion cohorts.'
]

summary_path = os.path.join('reports','summary.md')
with open(summary_path, 'a', encoding='utf-8') as f:
    f.write('\n\n# Business Recommendations\n\n')
    for r in recs:
        f.write(f'- {r}\n')
print('• ' + '\n• '.join(recs))
print(f'✅ Recommendations appended to {summary_path}')


## Conclusion & Next Steps
**Summary:** The analysis confirms device- and time-sensitive engagement patterns and highlights
content-type differences (e.g., short documentaries). Tactical levers focus on mobile UX, time-aware
programming, and cohort-specific messaging.

**Next steps:**
1. Build a BI dashboard (Tableau/Power BI) to monitor completion and engagement by cohort.
2. Cluster users by behavior (session length, device mix) to drive targeted lifecycle campaigns.
3. Prototype a Streamlit app to explore what-if scenarios (content length vs. device vs. hour).

In [ ]:
summary_path = os.path.join('reports','summary.md')
with open(summary_path, 'a', encoding='utf-8') as f:
    f.write('\n\n# Conclusion & Next Steps\n\n')
    f.write('**Summary:** The analysis confirms device- and time-sensitive engagement patterns and highlights content-type differences.\\n')
    f.write('**Next steps:** Build BI dashboards, run user clustering, prototype a Streamlit what-if tool.\\n')

print('🏁 Run complete! Key outputs:')
print(' - images/content_type_boxplot.png (after running Cell 10)')
print(' - images/hourly_engagement_heatmap.png (after running Cell 11)')
print(' - reports/summary.md (after running Cells 14, 16, 18)')
